# Compound precision loss vulnerability scanning

## Get listed markets
```sql
SELECT DISTINCT contract_address as comptroller, '0x' || SUBSTRING(data, -40) as market
FROM arbitrum.core.fact_event_logs
WHERE topics[0] = '0xcf583bb0c569eb967f806b11601c4cb93c10310485c67add5f8362c2f212321f' /* MarketListed(address) */
```

## Find markets with totalSupply = 0

In [42]:
import os
import pandas as pd
from web3 import Web3
from tqdm import tqdm
from dotenv import load_dotenv

data = pd.read_csv('arbitrum.csv')
data['TOTAL_SUPPLY'] = None
data['COLATERAL_FACTOR'] = None

load_dotenv()

rpc_url = os.getenv('ETH_RPC_URL', 'https://rpc.ankr.com/arbitrum')
w3 = Web3(Web3.HTTPProvider(rpc_url))

for index, row in tqdm(data.iterrows(), desc="Processing contracts", total=data.shape[0]):
    market = row.get("MARKET")
    if market == '0x':
        data.at[index, 'TOTAL_SUPPLY'] = None
        continue
    market = w3.to_checksum_address(market)
    contract = w3.eth.contract(
        address=market, 
        abi=[{
            "constant": True,
            "inputs": [],
            "name": "totalSupply",
            "outputs": [{"name": "", "type": "uint256"}],
            "payable": False,
            "stateMutability": "view",
            "type": "function"
        }]
    )
    
    try:
        total_supply = contract.functions.totalSupply().call()
    except Exception as e:
        print(f"Error calling totalSupply: {e}")
        data.at[index, 'TOTAL_SUPPLY'] = None
        continue

    data.at[index, 'TOTAL_SUPPLY'] = total_supply

empty_markets = data[data['TOTAL_SUPPLY'] == 0]
print("Empty markets: ", len(empty_markets))

Processing contracts: 100%|██████████| 757/757 [01:31<00:00,  8.27it/s]

Empty markets:  345


## Find markets with collateralFactor > 0

In [43]:
for index, item in tqdm(empty_markets.iterrows(), desc="Processing empty markets", total=empty_markets.shape[0]):
    comptroller_address = w3.to_checksum_address(item['COMPTROLLER'])
    comptroller_variants = [
        {
            "constant": True,
            "inputs": [{"name": "market", "type": "address"}],
            "name": "markets",
            "outputs": [
                {"name": "isListed", "type": "bool"},
                {"name": "collateralFactorMantissa", "type": "uint256"},
                {"name": "liquidationThresholdMantissa", "type": "uint256"},
                {"name": "collateralFactorMantissaVip", "type": "uint256"},
                {"name": "liquidationThresholdMantissaVip", "type": "uint256"},
                {"name": "isComped", "type": "bool"},
                {"name": "isPrivate", "type": "bool"},
                {"name": "onlyWhitelistedBorrow", "type": "bool"}
            ],
            "payable": False,
            "stateMutability": "view",
            "type": "function"
        },
        {
            "constant": True,
            "inputs": [{"name": "market", "type": "address"}],
            "name": "markets",
            "outputs": [
                {"name": "isListed", "type": "bool"},
                {"name": "collateralFactorMantissa", "type": "uint256"},
                {"name": "isComped", "type": "bool"}
            ],
            "payable": False,
            "stateMutability": "view",
            "type": "function"
        },
        {
            "constant": True,
            "inputs": [{"name": "market", "type": "address"}],
            "name": "markets",
            "outputs": [
                {"name": "isListed", "type": "bool"},
                {"name": "collateralFactorMantissa", "type": "uint256"}
            ],
            "payable": False,
            "stateMutability": "view",
            "type": "function"
        }
    ]
    
    for variant in comptroller_variants:
        try:
            comptroller = w3.eth.contract(
                address=comptroller_address, 
                abi=[variant]
            )
            market_info = comptroller.functions.markets(w3.to_checksum_address(item["MARKET"])).call()
            break
        except Exception as e:
            continue
    else:
        continue
    if market_info[1] > 0:
        #print("[Colateral Factor] market address: ", item["MARKET"], "comptroller address: ", item['COMPTROLLER'], "colfactor: ", market_info[1])
        data.at[index, 'COLATERAL_FACTOR'] = market_info[1]

non_zero_colfactor_markets = data[data['COLATERAL_FACTOR'] > 0]
print("Non-zero collateral factor markets: ", len(non_zero_colfactor_markets))


Processing empty markets: 100%|██████████| 345/345 [01:38<00:00,  3.50it/s]

Non-zero collateral factor markets:  241


## Calculate cash in comptrollers

In [44]:
comptroller_abi = [{
    "constant": True,
    "inputs": [],
    "name": "getAllMarkets",
    "outputs": [{"name": "", "type": "address[]"}],
    "payable": False,
    "stateMutability": "view",
    "type": "function"
}]

market_abi = [{
    "constant": True,
    "inputs": [],
    "name": "underlying",
    "outputs": [{"name": "", "type": "address"}],
    "payable": False,
    "stateMutability": "view",
    "type": "function"
},
{
    "constant": True,
    "inputs": [],
    "name": "getCash",
    "outputs": [{"name": "", "type": "uint256"}],
    "payable": False,
    "stateMutability": "view",
    "type": "function"
}]


underlying_abi = [{
    "constant": True,
    "inputs": [],
    "name": "decimals",
    "outputs": [{"name": "", "type": "uint8"}],
    "payable": False,
    "stateMutability": "view",
    "type": "function"
}, {
    "constant": True,
    "inputs": [],
    "name": "symbol",
    "outputs": [{"name": "", "type": "string"}],
    "payable": False,
    "stateMutability": "view",
    "type": "function"
}]


non_zero_colfactor_markets.drop_duplicates(subset=['COMPTROLLER'], keep='first', inplace=True)
comptrollers_df = pd.DataFrame(columns=['COMPTROLLER', 'MARKET', 'SYMBOL', 'CASH'])

for _, row in tqdm(non_zero_colfactor_markets.iterrows(), desc="Processing markets", total=non_zero_colfactor_markets.shape[0]):
    comptroller_address = w3.to_checksum_address(row['COMPTROLLER'])
    comptroller = w3.eth.contract(address=comptroller_address, abi=comptroller_abi)
    all_markets = comptroller.functions.getAllMarkets().call()

    for address in all_markets:
        try:
            market = w3.eth.contract(address=address, abi=market_abi)
            cash = market.functions.getCash().call()
            underlying_address = market.functions.underlying().call()
            underlying = w3.eth.contract(address=underlying_address, abi=underlying_abi)
            decimals = underlying.functions.decimals().call()
            symbol = underlying.functions.symbol().call()
            cash_in_market = cash / (10 ** decimals)

            if cash_in_market > 0:
                new_row = pd.DataFrame({'COMPTROLLER': [comptroller_address], 'MARKET': [address], 'SYMBOL': [symbol], 'CASH': [cash_in_market]})
                comptrollers_df = pd.concat([comptrollers_df, new_row], ignore_index=True)
        except Exception as e:
            continue

    current_comptroller_markets = comptrollers_df[comptrollers_df['COMPTROLLER'] == comptroller_address]
    
    if len(current_comptroller_markets) > 0:
        print(f"Comptroller: {comptroller_address}")
        for _, row in current_comptroller_markets.iterrows():
            print(f"Market: {row['MARKET']} {row['CASH']} {row['SYMBOL']}")


print("Comptrollers with cash: ", len(comptrollers_df['COMPTROLLER'].unique()))



/var/folders/4h/xq82s8553jzfpl6bxt6nstk00000gn/T/ipykernel_41918/4247130487.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_zero_colfactor_markets.drop_duplicates(subset=['COMPTROLLER'], keep='first', inplace=True)
Processing markets:   1%|▏         | 1/77 [00:02<03:10,  2.50s/it]

Comptroller: 0xCEaFD4e9CAcC070AF99CCE086C27AEDa837F8bd4
Market: 0x918368A637F6906002660cd6C5d3642d6cB6321C 0.2 USDT


Processing markets:   3%|▎         | 2/77 [00:09<06:16,  5.02s/it]

Comptroller: 0x49Ea2c991290cA13f57Ae2b8ca98bC6140925db3
Market: 0x1446f9365361Dbca6d7fF94852e9383Bf3BCc7E9 1e-18 fsGLP
Market: 0x038A8c9B8f25Fdb6d306FE1F4D99Ad9B3b3Ab722 6.9938753008483e-05 FRAX
Market: 0x102517Ea9340eDd21afdfAA911560311FeEFc607 0.013166 USDT
Market: 0xB1087a450373BB26BCf1A18E788269bde9c8fc85 0.478472 USDC
Market: 0xa2F156bc04329f9D0C71804a5B56f130674F44c9 0.382040350032375 fsGLP
Market: 0xD0af14CEe85b6612C02F75F35E90fE1f8e0fB2fD 3.6413875374481943 DAI
Market: 0xB88A54C70bC3A984039E0AfEc65A27F05812A7fB 1.0000000001e-08 GMX
Market: 0xE9d91a3E4A0A929E9A34510d44dFD335F39e17cB 0.1 fsGLP


Processing markets:   6%|▋         | 5/77 [00:19<04:23,  3.66s/it]

...

Comptroller: 0x18579B566019F85654C59C3b802d6e8F3B178B7b
Market: 0xd861026A12623aec769fA57D05201193D8844368 4.47939764923e-07 gOHM


Processing markets:  99%|█████████▊| 76/77 [03:05<00:01,  1.68s/it]

Comptroller: 0xbEbeF25702c845aAdee25DCf9a63D248C8fe5DAC
Market: 0xdFDb77841fD623aeC83D74df94E82dD073efCFb1 1.031967 USDT
Market: 0xc67DCCC71d3F9122ABb7Bfa38c6eA7120ED9bD65 1.000012 USDC
Market: 0x242f9347f485fDf8feF82e1B1de72b5C27F7eBcF 1e-18 fsGLP


Processing markets: 100%|██████████| 77/77 [03:06<00:00,  2.43s/it]

Comptroller: 0xD3a8ed9Cf0Ef1Bd677C6Ae7a2178837BED35aeBD
Market: 0xE728c318A6fD3b605726E810C2c542FD8056708c 2.5 plvGLP
Market: 0xF17C9D4062e2e652f81E43f9304EdDE3Fb3e63dB 1.3 USDC
Comptrollers with cash:  39
